In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import pickle
import pandas as pd
import random
import datetime as dt
import time

pd.set_option('mode.chained_assignment', None)
max_vel=500;
x_city_min = 3750901.5068
x_city_max = 3770901.5068
y_city_min =-19268905.6133
y_city_max =-19208905.6133

column_names = ['','hash','trajectory_id','time_entry','time_exit','vmax','vmin','vmean','x_entry','y_entry','x_exit','y_exit']
col_types = {'x_entry': np.float64}

df = pd.read_csv("data_test.csv", names=column_names,  skiprows=1, dtype = col_types)

print(df.head())
df.describe()
#df.dtypes



                                     hash  \
0  0  00032f51796fd5437b238e3a9823d13d_31   
1  1  00032f51796fd5437b238e3a9823d13d_31   
2  2  00032f51796fd5437b238e3a9823d13d_31   
3  3  00032f51796fd5437b238e3a9823d13d_31   
4  4  00032f51796fd5437b238e3a9823d13d_31   

                                trajectory_id time_entry time_exit  vmax  \
0  traj_00032f51796fd5437b238e3a9823d13d_31_0   11:43:17  11:50:17   NaN   
1  traj_00032f51796fd5437b238e3a9823d13d_31_2   12:21:37  12:21:37   0.0   
2  traj_00032f51796fd5437b238e3a9823d13d_31_3   12:34:27  13:14:11   NaN   
3  traj_00032f51796fd5437b238e3a9823d13d_31_4   13:25:33  13:43:13   NaN   
4  traj_00032f51796fd5437b238e3a9823d13d_31_5   15:03:32  15:10:32   NaN   

   vmin  vmean       x_entry       y_entry        x_exit        y_exit  
0   NaN    NaN  3.773413e+06 -1.909828e+07  3.773111e+06 -1.914508e+07  
1   0.0    0.0  3.773199e+06 -1.914354e+07  3.773199e+06 -1.914354e+07  
2   NaN    NaN  3.763760e+06 -1.921342e+07  3.771757e

,,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
count,202937.000000,54705.000000,54705.000000,57359.000000,2.029370e+05,2.029370e+05,1.694220e+05,1.694220e+05
mean,101468.000000,2.821230,2.421897,2.595662,3.760384e+06,-1.922144e+07,3.760393e+06,-1.922188e+07
std,58583.010127,7.782798,7.278356,7.332818,8.976732e+03,7.670937e+04,8.840276e+03,7.487858e+04
min,0.000000,-1.000000,-1.000000,-1.000000,3.741032e+06,-1.937675e+07,3.741020e+06,-1.937675e+07
25%,50734.000000,0.000000,0.000000,0.000000,3.755199e+06,-1.927516e+07,3.755436e+06,-1.927261e+07
50%,101468.000000,0.000000,0.000000,0.000000,3.760110e+06,-1.923015e+07,3.760101e+06,-1.923040e+07
75%,152202.000000,0.570000,0.280000,0.508373,3.767453e+06,-1.916959e+07,3.767306e+06,-1.917234e+07
max,202936.000000,153.000000,124.000000,124.000000,3.776975e+06,-1.904574e+07,3.777004e+06,-1.904699e+07


In [4]:
#remove velocity columns
df2 = df.drop(columns = ["","vmin", "vmax", "vmean"])
#df2 = df2[1:]

#print(df2.head(10))

In [6]:
#convert to seconds



df2['time_entry'] = pd.to_timedelta(df2['time_entry']).dt.total_seconds()

df2['time_exit'] = pd.to_timedelta(df2['time_exit']).dt.total_seconds()

#df3['entry_seconds'] = df3['time_entry'].dt.total_seconds()
#df3['exit_seconds'] = df3['exit_dtime'].dt.total_seconds()

#print(df2.head())

                                  hash  \
0  00032f51796fd5437b238e3a9823d13d_31   
1  00032f51796fd5437b238e3a9823d13d_31   
2  00032f51796fd5437b238e3a9823d13d_31   
3  00032f51796fd5437b238e3a9823d13d_31   
4  00032f51796fd5437b238e3a9823d13d_31   

                                trajectory_id  time_entry  time_exit  \
0  traj_00032f51796fd5437b238e3a9823d13d_31_0     42197.0    42617.0   
1  traj_00032f51796fd5437b238e3a9823d13d_31_2     44497.0    44497.0   
2  traj_00032f51796fd5437b238e3a9823d13d_31_3     45267.0    47651.0   
3  traj_00032f51796fd5437b238e3a9823d13d_31_4     48333.0    49393.0   
4  traj_00032f51796fd5437b238e3a9823d13d_31_5     54212.0    54632.0   

        x_entry       y_entry        x_exit        y_exit  
0  3.773413e+06 -1.909828e+07  3.773111e+06 -1.914508e+07  
1  3.773199e+06 -1.914354e+07  3.773199e+06 -1.914354e+07  
2  3.763760e+06 -1.921342e+07  3.771757e+06 -1.911092e+07  
3  3.773385e+06 -1.911344e+07  3.773131e+06 -1.914465e+07  
4  3.773118e+0

In [7]:
df2_last = df2.drop_duplicates(subset='hash', keep='last')
print(df2_last.head())

                                   hash  \
4   00032f51796fd5437b238e3a9823d13d_31   
7   000479418b5561ab694a2870cc04fd43_25   
10  000506a39775e5bca661ac80e3f466eb_29   
13   0005401ceddaf27a9b7f0d42ef1fbe95_1   
17   00063a4f6c12e1e4de7d876580620667_3   

                                  trajectory_id  time_entry  time_exit  \
4    traj_00032f51796fd5437b238e3a9823d13d_31_5     54212.0    54632.0   
7   traj_000479418b5561ab694a2870cc04fd43_25_10     55749.0    55856.0   
10   traj_000506a39775e5bca661ac80e3f466eb_29_5     55568.0    55568.0   
13    traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4     56118.0    56525.0   
17    traj_00063a4f6c12e1e4de7d876580620667_3_4     53647.0    54314.0   

         x_entry       y_entry  x_exit  y_exit  
4   3.773118e+06 -1.914490e+07     NaN     NaN  
7   3.769978e+06 -1.934136e+07     NaN     NaN  
10  3.757468e+06 -1.923860e+07     NaN     NaN  
13  3.760505e+06 -1.935500e+07     NaN     NaN  
17  3.766319e+06 -1.917013e+07     NaN     NaN  


In [8]:
# separate real trips
df2_last['deltaT']=df2_last['time_exit']-df2_last['time_entry']
df2_last['real_trip']=df2_last['deltaT']>0

#print(df3.head())


In [16]:
#Dataframe of last trips which are real

#df3=df2_last[df2_last['real_trip']]
#print(df3.head())
#df3.describe()






                                   hash  \
4   00032f51796fd5437b238e3a9823d13d_31   
7   000479418b5561ab694a2870cc04fd43_25   
13   0005401ceddaf27a9b7f0d42ef1fbe95_1   
17   00063a4f6c12e1e4de7d876580620667_3   
39  000d479078af08618bddc7f09082b8c3_11   

                                  trajectory_id  time_entry  time_exit  \
4    traj_00032f51796fd5437b238e3a9823d13d_31_5     54212.0    54632.0   
7   traj_000479418b5561ab694a2870cc04fd43_25_10     55749.0    55856.0   
13    traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4     56118.0    56525.0   
17    traj_00063a4f6c12e1e4de7d876580620667_3_4     53647.0    54314.0   
39   traj_000d479078af08618bddc7f09082b8c3_11_6     53886.0    54220.0   

         x_entry       y_entry  x_exit  y_exit  deltaT  real_trip  
4   3.773118e+06 -1.914490e+07     NaN     NaN   420.0       True  
7   3.769978e+06 -1.934136e+07     NaN     NaN   107.0       True  
13  3.760505e+06 -1.935500e+07     NaN     NaN   407.0       True  
17  3.766319e+06 -1.9170

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,deltaT
count,16674.000000,16674.000000,1.667400e+04,1.667400e+04,0.0,0.0,16674.000000
mean,54693.773240,55525.477030,3.760900e+06,-1.922078e+07,NaN,NaN,831.703790
std,1223.288682,1006.004756,9.261756e+03,7.624882e+04,NaN,NaN,792.014151
min,42157.000000,54000.000000,3.741041e+06,-1.937662e+07,NaN,NaN,1.000000
25%,53926.250000,54661.000000,3.755791e+06,-1.927472e+07,NaN,NaN,312.000000
50%,54605.000000,55395.000000,3.760997e+06,-1.922933e+07,NaN,NaN,595.000000
75%,55531.000000,56330.000000,3.767995e+06,-1.916987e+07,NaN,NaN,1108.000000
max,57541.000000,57600.000000,3.776969e+06,-1.904712e+07,NaN,NaN,14127.000000


In [10]:
df_fake_trips=df2_last[ ~(df2_last['real_trip'])]
print(df_fake_trips.head())
df_fake_trips.describe()

                                   hash  \
10  000506a39775e5bca661ac80e3f466eb_29   
23   0006535be25bb52dd06983447880c964_5   
27  0006f84bb33ec929d1cda7686f861d0a_31   
31  00093ae562586aed0e053b8431e8ace4_23   
34  000c739e444a70e1804d757a0580caaa_31   

                                  trajectory_id  time_entry  time_exit  \
10   traj_000506a39775e5bca661ac80e3f466eb_29_5     55568.0    55568.0   
23   traj_0006535be25bb52dd06983447880c964_5_12     55326.0    55326.0   
27   traj_0006f84bb33ec929d1cda7686f861d0a_31_3     54012.0    54012.0   
31  traj_00093ae562586aed0e053b8431e8ace4_23_10     57189.0    57189.0   
34   traj_000c739e444a70e1804d757a0580caaa_31_3     56491.0    56491.0   

         x_entry       y_entry  x_exit  y_exit  deltaT  real_trip  
10  3.757468e+06 -1.923860e+07     NaN     NaN     0.0      False  
23  3.776037e+06 -1.916993e+07     NaN     NaN     0.0      False  
27  3.760009e+06 -1.923361e+07     NaN     NaN     0.0      False  
31  3.756665e+06 -1.9146

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,deltaT
count,16841.000000,16841.000000,1.684100e+04,1.684100e+04,0.0,0.0,16841.0
mean,55500.453833,55500.453833,3.759976e+06,-1.922312e+07,NaN,NaN,0.0
std,1027.626347,1027.626347,8.979902e+03,7.659242e+04,NaN,NaN,0.0
min,54000.000000,54000.000000,3.741047e+06,-1.937673e+07,NaN,NaN,0.0
25%,54606.000000,54606.000000,3.754370e+06,-1.927849e+07,NaN,NaN,0.0
50%,55341.000000,55341.000000,3.759681e+06,-1.923123e+07,NaN,NaN,0.0
75%,56333.000000,56333.000000,3.767091e+06,-1.917305e+07,NaN,NaN,0.0
max,57600.000000,57600.000000,3.776975e+06,-1.904724e+07,NaN,NaN,0.0


In [11]:
#calc in city
####### should check this ########

df_fake_trips['incity']=(df_fake_trips['x_entry']>=x_city_min)&(df_fake_trips['x_entry']<=x_city_max)&(df_fake_trips['y_entry']>=y_city_min)&(df_fake_trips['y_entry']<=y_city_max)
print(df_fake_trips.head())
df_fake_trips.incity.value_counts()

                                   hash  \
10  000506a39775e5bca661ac80e3f466eb_29   
23   0006535be25bb52dd06983447880c964_5   
27  0006f84bb33ec929d1cda7686f861d0a_31   
31  00093ae562586aed0e053b8431e8ace4_23   
34  000c739e444a70e1804d757a0580caaa_31   

                                  trajectory_id  time_entry  time_exit  \
10   traj_000506a39775e5bca661ac80e3f466eb_29_5     55568.0    55568.0   
23   traj_0006535be25bb52dd06983447880c964_5_12     55326.0    55326.0   
27   traj_0006f84bb33ec929d1cda7686f861d0a_31_3     54012.0    54012.0   
31  traj_00093ae562586aed0e053b8431e8ace4_23_10     57189.0    57189.0   
34   traj_000c739e444a70e1804d757a0580caaa_31_3     56491.0    56491.0   

         x_entry       y_entry  x_exit  y_exit  deltaT  real_trip  incity  
10  3.757468e+06 -1.923860e+07     NaN     NaN     0.0      False    True  
23  3.776037e+06 -1.916993e+07     NaN     NaN     0.0      False   False  
27  3.760009e+06 -1.923361e+07     NaN     NaN     0.0      False   

False    11980
True      4861
Name: incity, dtype: int64

In [12]:
df_Y_test_fake_Trips=df_fake_trips[['trajectory_id','incity']]
print(df_Y_test_fake_Trips.head())

                                  trajectory_id  incity
10   traj_000506a39775e5bca661ac80e3f466eb_29_5    True
23   traj_0006535be25bb52dd06983447880c964_5_12   False
27   traj_0006f84bb33ec929d1cda7686f861d0a_31_3    True
31  traj_00093ae562586aed0e053b8431e8ace4_23_10   False
34   traj_000c739e444a70e1804d757a0580caaa_31_3    True


In [ ]:
#df3 = df3.drop(columns = ["entry_dtime","exit_dtime","entry_seconds","exit_seconds"])

In [ ]:
#Calculate velocity and remove >500

#df4['delta_X']=np.sqrt((df4['x_exit']-df4['x_entry'])**2+(df4['y_exit']-df4['y_entry'])**2)
#df4['vel']=df4['delta_X']/df4['deltaT']
#df4['reasonable_vel']=df4['vel']<max_vel
#df4=df4[df4['reasonable_vel']]
#print(df4.head())

In [ ]:
#calc in city

#df4['incity']=(df4['x_exit']>=x_city_min)&(df4['x_exit']<=x_city_max)&(df4['y_exit']>=y_city_min)&(df4['y_exit']<=y_city_max)
#print(df4.head())
#df4.incity.value_counts()

In [13]:

#load the pickle
Norm_Values = pickle.load(open("normvalues.pickle","rb"))
print(Norm_Values)


{'X_min': 3740998.35481912, 'X_max': 3777099.2656833767, 'Y_min': -19382914.9809002, 'Y_max': -19042656.658487003, 'T_min': 0.0, 'T_max': 57600}


In [17]:
#Remove devices with a fake last trip from the test data df2

df2['Good_hash']=~df2['hash'].isin(df_fake_trips['hash'])


df4=df2[df2['Good_hash']]
#df3.head()

In [30]:
#Normalize data

df4['Xentry_norm'] = (df4['x_entry']-Norm_Values['X_min'])/(Norm_Values['X_max'] - Norm_Values['X_min'])
df4['Xexit_norm'] = (df4['x_exit']-Norm_Values['X_min'])/(Norm_Values['X_max'] - Norm_Values['X_min'])

df4['Yentry_norm'] = (df4['y_entry']-Norm_Values['Y_min'])/(Norm_Values['Y_max'] - Norm_Values['Y_min'])
df4['Yexit_norm'] = (df4['y_exit']-Norm_Values['Y_min'])/(Norm_Values['Y_max'] - Norm_Values['Y_min'])

df4['T_entry_norm'] = (df4['time_entry'] - Norm_Values['T_min'])/ (Norm_Values['T_max'] - Norm_Values['T_min'])
df4['T_exit_norm'] = (df4['time_exit'] - Norm_Values['T_min'])/ (Norm_Values['T_max'] - Norm_Values['T_min'])


#print(df_x_test.head())


In [19]:
#Create a copy which will be for first trips only

df4_first = df4.copy()
#df4_first.describe()

In [20]:
#Drop duplicates keeping only the first trip for each hash

df4_first = df4_first.drop_duplicates(subset='hash', keep='first')
df4_first.head()

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,42197.0,42617.0,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,True,0.897903,0.889516,0.836524,0.698987,0.732587,0.739878
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,29303.0,30008.0,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07,True,0.841566,0.801480,0.147474,0.123338,0.508733,0.520972
11,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_0,33003.0,33003.0,3.751328e+06,-1.916236e+07,3.751328e+06,-1.916236e+07,True,0.286124,0.286124,0.648209,0.648209,0.572969,0.572969
14,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_1,31718.0,34436.0,3.747364e+06,-1.927846e+07,3.774308e+06,-1.911005e+07,True,0.176318,0.922695,0.306978,0.801938,0.550660,0.597847
35,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_0,49352.0,50110.0,3.744857e+06,-1.929530e+07,3.744840e+06,-1.929690e+07,True,0.106876,0.106414,0.257495,0.252800,0.856806,0.869965


In [21]:
#Drop duplicates keeping only the last trip for each hash

df4_last = df4.copy()
df4_last = df4_last.drop_duplicates(subset='hash', keep='last')
df4_last.head()

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212.0,54632.0,3.773118e+06,-1.914490e+07,NaN,NaN,True,0.889732,NaN,0.699514,NaN,0.941181,0.948472
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,55749.0,55856.0,3.769978e+06,-1.934136e+07,NaN,NaN,True,0.802744,NaN,0.122113,NaN,0.967865,0.969722
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,56118.0,56525.0,3.760505e+06,-1.935500e+07,NaN,NaN,True,0.540333,NaN,0.082031,NaN,0.974271,0.981337
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,53647.0,54314.0,3.766319e+06,-1.917013e+07,NaN,NaN,True,0.701388,NaN,0.625369,NaN,0.931372,0.942951
39,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,53886.0,54220.0,3.766505e+06,-1.920176e+07,NaN,NaN,True,0.706537,NaN,0.532399,NaN,0.935521,0.941319


In [24]:
#Identify duplicates except for the last one

df4['has_dup']= df4['hash'].duplicated(keep=False)
df4['is_last'] = df4['trajectory_id'].isin(df4_last['trajectory_id'])

#df4.head(10)

#Remove the last duplicate (only if there ARE duplicates)
df4_2last = df4.copy()
df4_2last = df4_2last[~((df4_2last['has_dup'])&(df4_2last['is_last']))]
#df4_2last.head(10)

#Remove duplicates keeping only the last one (really the second to last)
df4_2last = df4_2last.drop_duplicates(subset='hash', keep='last')
df4_2last.head(10)

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm,has_dup,is_last
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,48333.0,49393.0,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,True,0.897101,0.890071,0.791960,0.700240,0.839115,0.857517,True,False
6,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_6,43703.0,44004.0,3.771380e+06,-1.933274e+07,3.769983e+06,-1.934265e+07,True,0.841566,0.802868,0.147474,0.118337,0.758733,0.763958,True,False
12,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_1,34659.0,34659.0,3.751349e+06,-1.916284e+07,3.751349e+06,-1.916284e+07,True,0.286710,0.286710,0.646786,0.646786,0.601719,0.601719,True,False
16,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_3,51500.0,52502.0,3.776264e+06,-1.918289e+07,3.766296e+06,-1.917029e+07,True,0.976873,0.700740,0.587871,0.624905,0.894097,0.911493,True,False
38,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_4,52559.0,53826.0,3.744812e+06,-1.932195e+07,3.767684e+06,-1.920421e+07,True,0.105643,0.739192,0.179178,0.525208,0.912483,0.934479,True,False
55,000efd6f95db39d6ba35027f02b5628f_9,traj_000efd6f95db39d6ba35027f02b5628f_9_19,52893.0,52893.0,3.757572e+06,-1.923521e+07,3.757572e+06,-1.923521e+07,True,0.459081,0.459081,0.434084,0.434084,0.918281,0.918281,True,False
62,000feace3a33be9245e2783cdff467a8_15,traj_000feace3a33be9245e2783cdff467a8_15_10,52354.0,53368.0,3.770966e+06,-1.920456e+07,3.772139e+06,-1.916980e+07,True,0.830096,0.862596,0.524176,0.626329,0.908924,0.926528,True,False
65,0011b48d0cf4f962833edd701afaf419_3,traj_0011b48d0cf4f962833edd701afaf419_3_3,53464.0,53945.0,3.763424e+06,-1.911022e+07,3.763436e+06,-1.910955e+07,True,0.621184,0.621523,0.801446,0.803415,0.928194,0.936545,True,False
68,001484cf0f956c316070be4b4c6352ff_29,traj_001484cf0f956c316070be4b4c6352ff_29_6,45616.0,46956.0,3.755488e+06,-1.911968e+07,3.758029e+06,-1.919013e+07,True,0.401357,0.471755,0.773632,0.566586,0.791944,0.815208,True,False
71,001d29060cfe35996cfeb141210ecbeb_9,traj_001d29060cfe35996cfeb141210ecbeb_9_1,40785.0,42331.0,3.772801e+06,-1.908995e+07,3.742387e+06,-1.924661e+07,True,0.880944,0.038452,0.861006,0.400595,0.708073,0.734913,True,False


In [25]:
print(df4_first.shape)
print(df4_2last.shape)
print(df4_last.shape)

(16674, 15)
(16674, 17)
(16674, 15)


In [26]:
#renaming columns

df4_first.rename(index=str, columns={'trajectory_id': 'traj_1', 
                                  'Xentry_norm': 'Xi1','Xexit_norm': 'Xf1', 
                                  'Yentry_norm': 'Yi1','Yexit_norm':'Yf1',
                                  'T_entry_norm':'Ti1', 'T_exit_norm':'Tf1'},inplace= True)

#df4_first.head()

df4_2last.rename(index=str, columns={'trajectory_id': 'traj_2', 
                                  'Xentry_norm': 'Xi2','Xexit_norm': 'Xf2', 
                                  'Yentry_norm': 'Yi2','Yexit_norm':'Yf2',
                                  'T_entry_norm':'Ti2', 'T_exit_norm':'Tf2'},inplace= True)

#df4_2last.head()

df4_last.rename(index=str, columns={'trajectory_id': 'traj_3', 
                                  'Xentry_norm': 'Xi3','Xexit_norm': 'Xf3', 
                                  'Yentry_norm': 'Yi3','Yexit_norm':'Yf3',
                                  'T_entry_norm':'Ti3', 'T_exit_norm':'Tf3'},inplace= True)

#df4_last.head()

In [31]:
#Merge df first and second to last

df5 = pd.merge(df4_first,df4_2last,on='hash')


#merge dfs with last
df6 = pd.merge(df5,df4_last,on='hash')


#print(df6.head())
#df6.shape

df_x_test=df6

In [32]:
#save as pickle

#save df_Y_test_fake_Trips
pickle_out = open("df_Y_test_fake_Trips.pickle", "wb") 
pickle.dump(df_Y_test_fake_Trips,pickle_out) 
pickle_out.close()

#save df_x_test
pickle_out = open("df_x_test.pickle", "wb") 
pickle.dump(df_x_test,pickle_out) 
pickle_out.close()


In [ ]:
#convert to numpy arrays

#columns_np = ['Xentry_norm','Yentry_norm','T_entry_norm','T_exit_norm']
#np_entry = df4[columns_np].values


#print(np_entry[0:5])

In [ ]:
# load model


In [ ]:
#make predictions


In [ ]:
#put predictions back into the DF (order shouldnt have changed)

In [ ]:
#make predictions about the fake trips